In [1]:
import c3d_model
import caffe_pb2 as caffe
import numpy as np
import h5py
import os

def reindex(x):

    if x.ndim != 5:
        print ("[Error] Input to reindex must be 5D nparray.")
        return None

    N = x.shape[0]
    C = x.shape[1]
    L = x.shape[2]
    H = x.shape[3]
    W = x.shape[4]
    y = np.zeros_like(x)
    for n in range(N):
        for c in range(C):
            for l in range(L):
                for h in range(H):
                    for w in range(W):
                        y[n, c, l, h, w] = x[n, c,
                                                   L - l - 1,
                                                   H - h - 1,
                                                   W - w - 1]
    return y

def convert_dense(w):
    # kernel: (8192, 4096): (512x1x4x4, 4096) -> (1x4x4x512, 4096)
    wo = np.zeros_like(w)
    for i in range(w.shape[1]):
        wi = np.squeeze(w[:,i])
        wo[:,i] = np.transpose(np.reshape(wi, (512,4,4)), (1, 2, 0)).flatten()
    return wo

def main():

    #dim_ordering = 'th'
    #dim_ordering = 'th'
    import keras.backend as K
    dim_ordering = K.image_dim_ordering()
    print ("[Info] image_dim_order (from default ~/.keras/keras.json)={}".format(
            dim_ordering))

    # get C3D model placeholder
    model = c3d_model.get_model(summary=True, backend=dim_ordering)

    # input caffe model
    caffe_model_filename = './models/conv3d_deepnetA_sport1m_iter_1900000'

    # output dir/files
    model_dir = './models'
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    output_model_filename = os.path.join(model_dir, 'sports1M_weights_{}.h5'.format(dim_ordering))
    output_json_filename = os.path.join(model_dir, 'sports1M_weights_{}.json'.format(dim_ordering))

    # read caffe model
    print ("-" * 19)
    print ("Reading model file={}...".format(caffe_model_filename))
    p = caffe.NetParameter()
    p.ParseFromString(open(caffe_model_filename, 'rb').read())

    params = []
    print ("-" * 19)
    print ("Converting model...")

    # read every conv/fc layer and append to "params" list
    for i in range(len(p.layers)):
        layer = p.layers[i]
        # skip non-conv/fc layers
        if 'conv' not in layer.name and 'fc' not in layer.name:
            continue
        print ("[Info] Massaging \"{}\" layer...".format(layer.name))
        weights_b = np.array(layer.blobs[1].data, dtype=np.float32)
        weights_p = np.array(layer.blobs[0].data, dtype=np.float32).reshape(
            layer.blobs[0].num,
            layer.blobs[0].channels,
            layer.blobs[0].length,
            layer.blobs[0].height,
            layer.blobs[0].width,
            )
        if 'conv' in layer.name:
            # theano vs tensorflow: https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py#L90-L115
            if dim_ordering == 'th':
                weights_p = reindex(weights_p)
            else:
                weights_p = np.transpose(weights_p, (2, 3, 4, 1, 0))
        elif 'fc' in layer.name:
            weights_p = weights_p[0, 0, 0, :, :].T
            if 'fc6' in layer.name:
                print("[Info] First FC layer after flattening layer needs "
                      "special care...")
                weights_p = convert_dense(weights_p)
        params.append([weights_p, weights_b])

    valid_layer_count = 0
    for layer_indx in range(len(model.layers)):
        layer_name = model.layers[layer_indx].name
        if 'conv' in layer_name or 'fc' in layer_name:
            print ("[Info] Transplanting \"{}\" layer...".format(layer_name))
            model.layers[layer_indx].set_weights(params[valid_layer_count])
            valid_layer_count += 1

    print ("-" * 19)
    print ("Saving pre-trained model weights as {}...".format(output_model_filename))
    model.save_weights(output_model_filename, overwrite=True)
    json_string = model.to_json()
    with open(output_json_filename, 'w') as f:
        f.write(json_string)
    print ("-" * 39)
    print ("Conversion done!")
    print ("-" * 39)

if __name__ == '__main__':
    main()


Using TensorFlow backend.


[Info] image_dim_order (from default ~/.keras/keras.json)=tf
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 16, 112, 112, 64)  5248      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 16, 56, 56, 64)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 16, 56, 56, 128)   221312    
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 8, 28, 28, 128)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 8, 28, 28, 256)    884992    
_________________________________________________________________
conv3b (Conv3D)              (None, 8, 28, 28, 256)    1769728   
_________________________________________________________________
pool3 (MaxPooli

/home/digvijay/Desktop/c3d-keras-master/c3d_model.py:22: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), input_shape=(16, 112, ..., name="conv1", padding="same", activation="relu")`
  input_shape=input_shape))
/home/digvijay/Desktop/c3d-keras-master/c3d_model.py:24: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(strides=(1, 2, 2), name="pool1", padding="valid", pool_size=(1, 2, 2))`
  border_mode='valid', name='pool1'))
/home/digvijay/Desktop/c3d-keras-master/c3d_model.py:27: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (3, 3, 3), name="conv2", padding="same", activation="relu")`
  border_mode='same', name='conv2'))
/home/digvijay/Desktop/c3d-keras-master/c3d_model.py:29: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(strides=(2, 2, 2), name="pool2", padding="valid", pool_size=(2, 2, 2))`
  border_mode='valid', name='pool2'))
/home/digvijay/Desktop/c3d-keras-mast

-------------------
Converting model...
[Info] Massaging "conv1a" layer...
[Info] Massaging "conv2a" layer...
[Info] Massaging "conv3a" layer...
[Info] Massaging "conv3b" layer...
[Info] Massaging "conv4a" layer...
[Info] Massaging "conv4b" layer...
[Info] Massaging "conv5a" layer...
[Info] Massaging "conv5b" layer...
[Info] Massaging "fc6-1" layer...
[Info] First FC layer after flattening layer needs special care...
[Info] Massaging "fc7-1" layer...
[Info] Massaging "fc8-1" layer...
[Info] Transplanting "conv1" layer...
[Info] Transplanting "conv2" layer...
[Info] Transplanting "conv3a" layer...
[Info] Transplanting "conv3b" layer...
[Info] Transplanting "conv4a" layer...
[Info] Transplanting "conv4b" layer...
[Info] Transplanting "conv5a" layer...
[Info] Transplanting "conv5b" layer...
[Info] Transplanting "fc6" layer...
[Info] Transplanting "fc7" layer...
[Info] Transplanting "fc8" layer...
-------------------
Saving pre-trained model weights as ./models/sports1M_weights_tf.h5...
--

In [2]:
#!/usr/bin/env python

import matplotlib
matplotlib.use('Agg')
from keras.models import model_from_json
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import c3d_model
import sys
import keras.backend as K
dim_ordering = K.image_dim_ordering()
print ("[Info] image_dim_order (from default ~/.keras/keras.json)={}".format(
        dim_ordering))
backend = dim_ordering

def diagnose(data, verbose=True, label='input', plots=False, backend='tf'):
    # Convolution3D?
    if data.ndim > 2:
        if backend == 'th':
            data = np.transpose(data, (1, 2, 3, 0))
        #else:
        #    data = np.transpose(data, (0, 2, 1, 3))
        min_num_spatial_axes = 10
        max_outputs_to_show = 3
        ndim = data.ndim
        print ("[Info] {}.ndim={}".format(label, ndim))
        print ("[Info] {}.shape={}".format(label, data.shape))
        for d in range(ndim):
            num_this_dim = data.shape[d]
            if num_this_dim >= min_num_spatial_axes: # check for spatial axes
                # just first, center, last indices
                range_this_dim = [0, num_this_dim/2, num_this_dim - 1]
            else:
                # sweep all indices for non-spatial axes
                range_this_dim = range(num_this_dim)
            for i in range_this_dim:
                new_dim = tuple([d] + range(d) + range(d + 1, ndim))
                sliced = np.transpose(data, new_dim)[i, ...]
                print("[Info] {}, dim:{} {}-th slice: "
                      "(min, max, mean, std)=({}, {}, {}, {})".format(
                              label,
                              d, i,
                              np.min(sliced),
                              np.max(sliced),
                              np.mean(sliced),
                              np.std(sliced)))
        if plots:
            # assume (l, h, w, c)-shaped input
            if data.ndim != 4:
                print("[Error] data (shape={}) is not 4-dim. Check data".format(
                        data.shape))
                return
            l, h, w, c = data.shape
            if l >= min_num_spatial_axes or \
                h < min_num_spatial_axes or \
                w < min_num_spatial_axes:
                print("[Error] data (shape={}) does not look like in (l,h,w,c) "
                      "format. Do reshape/transpose.".format(data.shape))
                return
            nrows = int(np.ceil(np.sqrt(data.shape[0])))
            # BGR
            if c == 3:
                for i in range(l):
                    mng = plt.get_current_fig_manager()
                    mng.resize(*mng.window.maxsize())
                    plt.subplot(nrows, nrows, i + 1) # doh, one-based!
                    im = np.squeeze(data[i, ...]).astype(np.float32)
                    im = im[:, :, ::-1] # BGR to RGB
                    # force it to range [0,1]
                    im_min, im_max = im.min(), im.max()
                    if im_max > im_min:
                        im_std = (im - im_min) / (im_max - im_min)
                    else:
                        print ("[Warning] image is constant!")
                        im_std = np.zeros_like(im)
                    plt.imshow(im_std)
                    plt.axis('off')
                    plt.title("{}: t={}".format(label, i))
                plt.show()
                #plt.waitforbuttonpress()
            else:
                for j in range(min(c, max_outputs_to_show)):
                    for i in range(l):
                        mng = plt.get_current_fig_manager()
                        mng.resize(*mng.window.maxsize())
                        plt.subplot(nrows, nrows, i + 1) # doh, one-based!
                        im = np.squeeze(data[i, ...]).astype(np.float32)
                        im = im[:, :, j]
                        # force it to range [0,1]
                        im_min, im_max = im.min(), im.max()
                        if im_max > im_min:
                            im_std = (im - im_min) / (im_max - im_min)
                        else:
                            print ("[Warning] image is constant!")
                            im_std = np.zeros_like(im)
                        plt.imshow(im_std)
                        plt.axis('off')
                        plt.title("{}: o={}, t={}".format(label, j, i))
                    plt.show()
                    #plt.waitforbuttonpress()
    elif data.ndim == 1:
        print("[Info] {} (min, max, mean, std)=({}, {}, {}, {})".format(
                      label,
                      np.min(data),
                      np.max(data),
                      np.mean(data),
                      np.std(data)))
        print("[Info] data[:10]={}".format(data[:10]))

    return

def main():
    show_images = True
    diagnose_plots = False
    model_dir = './models'
    global backend

    # override backend if provided as an input arg
    if len(sys.argv) > 1:
        if 'tf' in sys.argv[1].lower():
            backend = 'tf'
        else:
            backend = 'tf'
    print ("[Info] Using backend={}".format(backend))

    if backend == 'th':
        model_weight_filename = os.path.join(model_dir, 'sports1M_weights_tf.h5')
        model_json_filename = os.path.join(model_dir, 'sports1M_weights_tf.json')
    else:
        model_weight_filename = os.path.join(model_dir, 'sports1M_weights_tf.h5')
        model_json_filename = os.path.join(model_dir, 'sports1M_weights_tf.json')

    print("[Info] Reading model architecture...")
    model = model_from_json(open(model_json_filename, 'r').read())
    #model = c3d_model.get_model(backend=backend)

    # visualize model
    model_img_filename = os.path.join(model_dir, 'c3d_model.png')
    if not os.path.exists(model_img_filename):
        from keras.utils import plot_model
        plot_model(model, to_file=model_img_filename)

    print("[Info] Loading model weights...")
    model.load_weights(model_weight_filename)
    print("[Info] Loading model weights -- DONE!")
    model.compile(loss='mean_squared_error', optimizer='sgd')

    print("[Info] Loading labels...")
    with open('sports1m/labels.txt', 'r') as f:
        labels = [line.strip() for line in f.readlines()]
    print('Total labels: {}'.format(len(labels)))

    print("[Info] Loading a sample video...")
    cap = cv2.VideoCapture('dM06AMFLsrc.mp4')

    vid = []
    while True:
        ret, img = cap.read()
        if not ret:
            break
        vid.append(cv2.resize(img, (171, 128)))
    vid = np.array(vid, dtype=np.float32)

    #plt.imshow(vid[2000]/256)
    #plt.show()

    # sample 16-frame clip
    #start_frame = 100
    start_frame = 2000
    X = vid[start_frame:(start_frame + 16), :, :, :]
    #diagnose(X, verbose=True, label='X (16-frame clip)', plots=show_images)

    # subtract mean
    mean_cube = np.load('models/train01_16_128_171_mean.npy')
    mean_cube = np.transpose(mean_cube, (1, 2, 3, 0))
    #diagnose(mean_cube, verbose=True, label='Mean cube', plots=show_images)
    X -= mean_cube
    #diagnose(X, verbose=True, label='Mean-subtracted X', plots=show_images)

    # center crop
    X = X[:, 8:120, 30:142, :] # (l, h, w, c)
    #diagnose(X, verbose=True, label='Center-cropped X', plots=show_images)

    if backend == 'th':
        X = np.transpose(X, (3, 0, 1, 2)) # input_shape = (3,16,112,112)
    else:
        pass                              # input_shape = (16,112,112,3)

    # get activations for intermediate layers if needed
    inspect_layers = [
    #    'fc6',
    #    'fc7',
        ]
    for layer in inspect_layers:
        int_model = c3d_model.get_int_model(model=model, layer=layer, backend=backend)
        int_output = int_model.predict_on_batch(np.array([X]))
        int_output = int_output[0, ...]
        print ("[Debug] at layer={}: output.shape={}".format(layer, int_output.shape))
        diagnose(int_output,
                 verbose=True,
                 label='{} activation'.format(layer),
                 plots=diagnose_plots,
                 backend=backend)
    ress=np.array([X])
    # inference
    output = model.predict_on_batch(np.array([X]))

  
    print('test')
    plt.plot(output[0])
    plt.show()
    
    print('Position of maximum probability: {}'.format(output[0].argmax()))
    print('Maximum probability: {:.5f}'.format(max(output[0])))
    print('Corresponding label: {}'.format(labels[output[0].argmax()]))

  
    top_inds = output[0].argsort()[::-1][:5]  # reverse sort and take five largest items
    print('\nTop 5 probabilities and labels:')
    for i in top_inds:
        print('{1}: {0:.5f}'.format(output[0][i], labels[i]))

if __name__ == '__main__':
    main()


[Info] image_dim_order (from default ~/.keras/keras.json)=tf
[Info] Using backend=tf
[Info] Reading model architecture...
[Info] Loading model weights...
[Info] Loading model weights -- DONE!
[Info] Loading labels...
Total labels: 487
[Info] Loading a sample video...
test
Position of maximum probability: 133
Maximum probability: 0.97362
Corresponding label: golf

Top 5 probabilities and labels:
golf: 0.97362
match play: 0.02486
pitch and putt: 0.00147
one day international: 0.00003
cricket: 0.00001
